# Predykcja wyniku meczu CS:GO
CS:GO - https://pl.wikipedia.org/wiki/Counter-Strike:_Global_Offensive

### Dane:
Dane zostały pozyskane poprzez webscarping ogolnodostępnej strony : https://www.hltv.org/, zawierające wyniki oraz statystyki profesjonalnych meczów cs:go.
Skrypty potrzebne do pobrania danych znajduja się w osobnym pliku !!!!!!!!!!!!!(NAZWE WSTAW)

### Cel:
Okreslenie wyniku pojedynczego spotkania - która drużyna zwycięzy. Sprowadza się to do zadania 1 jezeli wygrywa pierwsza druzyna oraz 0 jezeli wygrywa 2 druzyna (klasyfikator binarny)

### Klasa problemu:
Jako że znamy wyniki spotkań(label) mozemy wykorzystać uczenie nadzorowane (supervised learning). Znana jest takze liczba klas (2)





In [1]:
import pandas as pd
import numpy as np

Pobieramy dane wczesniej przygotowane po stronie 2 skryptu

In [73]:
df_data = pd.read_pickle('df_data_connected.pkl')

In [74]:
pd.set_option('display.max_columns', 500)
df_data.head(10)

,date,team1main,team2main,map,eventmain,link,score_team1main,score_team2main,date_time,eventdetails,map_number,match_type,score_team1details,score_team2details,score_total,team1details,team1_clutches_won,team1_first_kills,team1_p_ADR_1,team1_p_ADR_2,team1_p_ADR_3,team1_p_ADR_4,team1_p_ADR_5,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,team1_p_death_1,team1_p_death_2,team1_p_death_3,team1_p_death_4,team1_p_death_5,team1_p_kast_1,team1_p_kast_2,team1_p_kast_3,team1_p_kast_4,team1_p_kast_5,team1_p_kills_1,team1_p_kills_2,team1_p_kills_3,team1_p_kills_4,team1_p_kills_5,team1_p_rating_1,team1_p_rating_2,team1_p_rating_3,team1_p_rating_4,team1_p_rating_5,team1_player_name_1,team1_player_name_2,team1_player_name_3,team1_player_name_4,team1_player_name_5,team1_rating,team1_score_1half,team1_score_2half,team1side,team2details,team2_clutches_won,team2_first_kills,team2_p_ADR_1,team2_p_ADR_2,team2_p_ADR_3,team2_p_ADR_4,team2_p_ADR_5,team2_p_asists_1,team2_p_asists_2,team2_p_asists_3,team2_p_asists_4,team2_p_asists_5,team2_p_death_1,team2_p_death_2,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kast_1,team2_p_kast_2,team2_p_kast_3,team2_p_kast_4,team2_p_kast_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_p_rating_1,team2_p_rating_2,team2_p_rating_3,team2_p_rating_4,team2_p_rating_5,team2_player_name_1,team2_player_name_2,team2_player_name_3,team2_player_name_4,team2_player_name_5,team2_rating,team2_score_1half,team2_score_2half,map_name
0,2012-10-11,NiP,HastaLaVista,Train_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,3,2012-10-11 20:00,ESWC 2012 Nordic Qualifier,2.0,Best of 3,16,3,2 - 0,NiP,1,14,-,-,-,-,-,0,0,0,0,0,7,9,4,11,8,-,-,-,-,-,23 (0),20 (0),14 (0),16 (0),12 (0),1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,HastaLaVista,0,5,-,-,-,-,-,0,0,0,0,0,17,17,16,18,18,-,-,-,-,-,11 (0),8 (0),6 (0),7 (0),7 (0),0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,Train_se
1,2012-10-11,NiP,HastaLaVista,Train_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,3,2012-10-11 20:00,ESWC 2012 Nordic Qualifier,1.0,Best of 3,16,8,2 - 0,NiP,2,15,-,-,-,-,-,0,0,0,0,0,13,15,14,15,16,-,-,-,-,-,26 (0),21 (0),18 (0),17 (0),16 (0),1.86,1.30,1.09,1.02,0.90,f0rest,friberg,GeT_RiGhT,Xizt,Fifflaren,1.23,10,6,ct-color,HastaLaVista,3,9,-,-,-,-,-,0,0,0,0,0,17,19,19,21,22,-,-,-,-,-,20 (0),16 (0),16 (0),10 (0),8 (0),1.32,0.92,0.82,0.45,0.37,Pimp,zn0pe,cajunb,ruggah,MSL,0.78,5,3,Train_se
2,2012-10-11,NiP,HastaLaVista,Inferno_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,8,2012-10-11 20:00,ESWC 2012 Nordic Qualifier,2.0,Best of 3,16,3,2 - 0,NiP,1,14,-,-,-,-,-,0,0,0,0,0,7,9,4,11,8,-,-,-,-,-,23 (0),20 (0),14 (0),16 (0),12 (0),1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,HastaLaVista,0,5,-,-,-,-,-,0,0,0,0,0,17,17,16,18,18,-,-,-,-,-,11 (0),8 (0),6 (0),7 (0),7 (0),0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0,Inferno_se
3,2012-10-11,NiP,HastaLaVista,Inferno_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,8,2012-10-11 20:00,ESWC 2012 Nordic Qualifier,1.0,Best of 3,16,8,2 - 0,NiP,2,15,-,-,-,-,-,0,0,0,0,0,13,15,14,15,16,-,-,-,-,-,26 (0),21 (0),18 (0),17 (0),16 (0),1.86,1.30,1.09,1.02,0.90,f0rest,friberg,GeT_RiGhT,Xizt,Fifflaren,1.23,10,6,ct-color,HastaLaVista,3,9,-,-,-,-,-,0,0,0,0,0,17,19,19,21,22,-,-,-,-,-,20 (0),16 (0),16 (0),10 (0),8 (0),1.32,0.92,0.82,0.45,0.37,Pimp,zn0pe,cajunb,ruggah,MSL,0.78,5,3,Inferno_se
4,2012-10-11,Red Dead,mousesports,Inferno_se,ESL Pro Series Germany Winter Season 2012,https://www.hltv.org/stats/matches/mapstatsid/...,2,16,2012-10-11 20:00,ESL Pro Series Germany Winter Season 2012,1.0,Best of 1,16,2,NaN,mousesports,3,13,-,-,-,-,-,0,0,0,0,0,6,11,11,9,10,-,-,-,-,-,24 (0),17 (0),16 (0),13 (0),12 (0),2.08,1.30,1.25,1.11,0.98

##### 1. Usuwamy zduplikowane wiersze

In [80]:
df_data = df_data.drop_duplicates(['link'])

##### Ponizej liczba wierszy przed wstepna obróbka, sprawdzamy typy naszych danych

In [83]:
len(df_data.index)

66654

In [86]:
df_data.dtypes

date                   datetime64[ns]
team1main                      object
team2main                      object
map                            object
eventmain                      object
link                           object
score_team1main                object
score_team2main                object
date_time                      object
eventdetails                   object
map_number                    float64
match_type                     object
score_team1details             object
score_team2details             object
score_total                    object
team1details                   object
team1_clutches_won             object
team1_first_kills              object
team1_p_ADR_1                  object
team1_p_ADR_2                  object
team1_p_ADR_3                  object
team1_p_ADR_4                  object
team1_p_ADR_5                  object
team1_p_asists_1               object
team1_p_asists_2               object
team1_p_asists_3               object
team1_p_asis

##### 2. Większosc naszych danych jest typu object - zamieniamy je do prawidlowego typu

Dane : 
- date - data spotkanie - te same informacje mamy w date_time - USUWAMY
- team1main - (str) nazwa zespol 1
- team2main - (str) nazwa zespol 2 
- map - (str) nazwa mapy 
- eventmain - (str) nazwa eventu 
- link - (str) link do strony zawierajace informacje o spotkaniu, bez wartosci przy modelowaniu - USUWAMY
- score_team1main - (int) wynik 1 zespol
- score_team2main - (int) wynik 2 zespol
- date_time - (datetime) data oraz czas spotkanie 
- eventdetails - (str) nazwa eventu - nazwa powtorzona - USUWAMY 
- map_number - (int) - nr mapy dla calego meczu 
- match_type - (str) - typ spotkania (best of 3, best of 5)
- score_team1details & score_team2details - nazwy druzyn - USUWAMY
- score_total -  wynik dla calego meczu - zawiera dane ktore nie beda dostepne w sytuacji prognozy(dane z przyszłosci)-USUWANY
- team1details & team2details - nazwy druzyn - juz obecne w bazie - usuwamy
- team1_clutches_won & team2_clutches_won - (int) wygranych clutchy - odpowiednio dla 1 i 2 druzyny (w sytuacji przewagi jednej z druzyn
- team1_first_kills & team2_first_kills - (int) liczba otwierajacych zabojstw
- team1_p_ADR_1 - (float) Average Damage Ratio - srednia zadawanych obrazen tu dla 1 gracza dla 1 druzyny - reszta ADR analogicznie 
- team1_p_asists_1 (int) liczba asyst w zabojstwach - analogicznie dla pozostalych graczy
- team1_p_death_1 (int) liczba smierci - analogicznie dla pozostalych graczy
- team1_p_kast_1 (float)  procent rund w ktorych gracz zaliczyl zabojstwo lub asyste - analogicznie dla pozostalych graczy
- team1_p_kills_1 (int) liczba zabojstw - analogicznie dla pozostalych graczy
- team1_p_rating_1 - (float) - rating dla gracza ustalany prze HLTV - analogicznie dla pozostalych graczy
- team1_player_name_1 (str) - nick zawodnika
- team1_rating (float) rating dla calego zespolu ustalany prze HLTV - analogicznie dla pozostalych graczy
- team1_score_1half (int) - wynik 1 zespolu po 1 polowwie - analogicznie dla 2 zespolu oraz dla 2 polow
- team1side (str) - strona po ktorej rozpoczyna 1 zespol
- map name - nazwa mapy - powtorzone - USUWAMY




In [88]:
# usuwamy niepotrzebne kolumny

#powtorzone kolumny
df_data = df_data.drop(columns =['eventdetails', 'score_team1details','score_team2details','team1details','team2details','map_name'])

#nie bedziemy mieli tej danej w przyszlosci
df_data = df_data.drop(columns =['score_total'])

In [91]:
# dodatkowo usuwamy wiersze dla ktorych nie udalo sie pobrac danych szczegolowych
df_data = df_data.dropna()

In [94]:
# mozemy takze zawazyc ze dane niedostepne sa zapisane jako - oznacza to ze dane zostaly pobrane ale nie sa one dostepne
# np. team1_p_ADR_1
df_data.head(5)

,date,team1main,team2main,map,eventmain,link,score_team1main,score_team2main,date_time,map_number,match_type,team1_clutches_won,team1_first_kills,team1_p_ADR_1,team1_p_ADR_2,team1_p_ADR_3,team1_p_ADR_4,team1_p_ADR_5,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,team1_p_death_1,team1_p_death_2,team1_p_death_3,team1_p_death_4,team1_p_death_5,team1_p_kast_1,team1_p_kast_2,team1_p_kast_3,team1_p_kast_4,team1_p_kast_5,team1_p_kills_1,team1_p_kills_2,team1_p_kills_3,team1_p_kills_4,team1_p_kills_5,team1_p_rating_1,team1_p_rating_2,team1_p_rating_3,team1_p_rating_4,team1_p_rating_5,team1_player_name_1,team1_player_name_2,team1_player_name_3,team1_player_name_4,team1_player_name_5,team1_rating,team1_score_1half,team1_score_2half,team1side,team2_clutches_won,team2_first_kills,team2_p_ADR_1,team2_p_ADR_2,team2_p_ADR_3,team2_p_ADR_4,team2_p_ADR_5,team2_p_asists_1,team2_p_asists_2,team2_p_asists_3,team2_p_asists_4,team2_p_asists_5,team2_p_death_1,team2_p_death_2,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kast_1,team2_p_kast_2,team2_p_kast_3,team2_p_kast_4,team2_p_kast_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_p_rating_1,team2_p_rating_2,team2_p_rating_3,team2_p_rating_4,team2_p_rating_5,team2_player_name_1,team2_player_name_2,team2_player_name_3,team2_player_name_4,team2_player_name_5,team2_rating,team2_score_1half,team2_score_2half
0,2012-10-11,NiP,HastaLaVista,Train_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,3,2012-10-11 20:00,2.0,Best of 3,1,14,-,-,-,-,-,0,0,0,0,0,7,9,4,11,8,-,-,-,-,-,23 (0),20 (0),14 (0),16 (0),12 (0),1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,0,5,-,-,-,-,-,0,0,0,0,0,17,17,16,18,18,-,-,-,-,-,11 (0),8 (0),6 (0),7 (0),7 (0),0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0
2,2012-10-11,NiP,HastaLaVista,Inferno_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,8,2012-10-11 20:00,2.0,Best of 3,1,14,-,-,-,-,-,0,0,0,0,0,7,9,4,11,8,-,-,-,-,-,23 (0),20 (0),14 (0),16 (0),12 (0),1.92,1.55,1.53,1.32,1.09,f0rest,Xizt,Fifflaren,friberg,GeT_RiGhT,1.49,12,4,ct-color,0,5,-,-,-,-,-,0,0,0,0,0,17,17,16,18,18,-,-,-,-,-,11 (0),8 (0),6 (0),7 (0),7 (0),0.69,0.47,0.45,0.44,0.38,ruggah,cajunb,Pimp,MSL,zn0pe,0.49,3,0
4,2012-10-11,Red Dead,mousesports,Inferno_se,ESL Pro Series Germany Winter Season 2012,https://www.hltv.org/stats/matches/mapstatsid/...,2,16,2012-10-11 20:00,1.0,Best of 1,3,13,-,-,-,-,-,0,0,0,0,0,6,11,11,9,10,-,-,-,-,-,24 (0),17 (0),16 (0),13 (0),12 (0),2.08,1.30,1.25,1.11,0.98,Troubley,stavros,approx,nex,strux1,1.34,13,3,t-color,0,4,-,-,-,-,-,0,0,0,0,0,17,16,17,17,16,-,-,-,-,-,18 (0),11 (0),7 (0),7 (0),4 (0),1.11,0.73,0.47,0.40,0.28,fl0w,cavy,stefan,kev1n,caLipo,0.60,2,0
5,2012-10-10,mousesports,Tt Dragons,Mirage_ce,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,8,2012-10-10 21:00,2.0,Best of 3,0,12,-,-,-,-,-,0,0,0,0,0,19,17,19,22,17,-,-,-,-,-,22 (14),17 (7),15 (4),16 (8),5 (2),1.15,0.97,0.76,0.67,0.48,Kaiii,mT,MoRs.na,kemo,deathX,0.81,6,2,t-color,2,12,-,-,-,-,-,0,0,0,0,0,16,14,18,13,14,-,-,-,-,-,23 (12),21 (11),20 (3),14 (2),16 (8),1.56,1.34,1.14,0.98,0.97,stavros,Troubley,strux1,approx,nex,1.20,9,7
7,2012-10-10,Tt Dragons,mousesports,Dust2_se,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,7,16,2012-10-10 21:00,2.0,Best of 3,0,12,-,-,-,-,-,0,0,0,0,0,19,17,19,22,17,-,-,-,-,-,22 (14),17 (7),15 (4),16 (8),5 (2),1.15,0.97,0.76,0.67,0.48,Kaiii,mT,MoRs.na,kemo,deathX,0.81,6,2,t-color,2,12,-,-,-,-,-,0,0,0,0,0,16,14,18,13,14,-,-,-,-,-,23 (12),21 (11),20 (3),14 (2),16 (8),1.56,1.34,1.14,0.98,0.97,stavros,Troubley,strux1,approx,nex,1.20,9,7


In [96]:
df_data = df_data.replace('-', np.nan)

In [99]:
# kolumna z zabojstwami zawiera informacje w formacie 12 (2) - z czym 1 wartosc oznacza liczbe zabojstw a 2 oznacza liczbe headshotow 
# przeksztalcamy dane w taki sposob aby wyciagnac z danych tylko liczbe zabojstw
df_data.team1_p_kills_1.iloc[-1]

'7 (1)'

In [100]:
def get_kills(row_text):
    kills = row_text.split(" ")[0]
    return kills

lista_kolumn = ['team1_p_kills_1', 'team1_p_kills_2',
       'team1_p_kills_3', 'team1_p_kills_4', 'team1_p_kills_5', 'team1_p_asists_1',
       'team1_p_asists_2', 'team1_p_asists_3', 'team1_p_asists_4',
       'team1_p_asists_5', 'team2_p_asists_1',
       'team2_p_asists_2', 'team2_p_asists_3', 'team2_p_asists_4',
       'team2_p_asists_5','team2_p_kills_1', 'team2_p_kills_2',
       'team2_p_kills_3', 'team2_p_kills_4', 'team2_p_kills_5']

for kolumna in lista_kolumn:
    df_data[kolumna] = df_data[kolumna].apply(lambda x: get_kills(x))

In [102]:
# kolumna z % rund w ktorych byla asysta lub zabojstwo - jest zapisana w formacie XX% - ktory jest formatem str - 
# wycinamy %
df_data.iloc[-1]['team1_p_kast_1']

'96.3%'

In [104]:
def remove_last_elment_from_str(cell):
    if isinstance(cell , str):
        cell=cell[:-1]
    return cell


column_list = ['team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5', 'team2_p_kast_1', 'team2_p_kast_2', 'team2_p_kast_3', 'team2_p_kast_4',
       'team2_p_kast_5']

for column in column_list:
    df_data[column] = df_data[column].apply(lambda x: remove_last_elment_from_str(x))

In [105]:
# mozemy zmienic typ danych na prawidlowe

In [ ]:
#niestety inplace nie dziala


# NAPRAW TA KONWERSJE NA STRING
# str_list = ['team1main','team2main','map','eventmain','link','match_type','team1_player_name_1',
#        'team1_player_name_2', 'team1_player_name_3', 'team1_player_name_4',
#        'team1_player_name_5','team1side','team2_player_name_1',
#        'team2_player_name_2', 'team2_player_name_3', 'team2_player_name_4',
#        'team2_player_name_5']


# def to_string(column):
#     return new_str = [str(x) for x in old_obj][0]

# for column in str_list:
#     df_data[column] = df_data[column].apply(str)



int_list = ['score_team1main' , 'score_team2main','map_number','team1_clutches_won', 'team1_first_kills','team1_p_asists_1',
       'team1_p_asists_2', 'team1_p_asists_3', 'team1_p_asists_4',
       'team1_p_asists_5', 'team1_p_death_1', 'team1_p_death_2',
       'team1_p_death_3', 'team1_p_death_4', 'team1_p_death_5','team1_p_kills_1', 'team1_p_kills_2',
       'team1_p_kills_3', 'team1_p_kills_4', 'team1_p_kills_5','team1_score_1half',
       'team1_score_2half', 'team2_clutches_won', 'team2_first_kills','team2_p_asists_1',
       'team2_p_asists_2', 'team2_p_asists_3', 'team2_p_asists_4',
       'team2_p_asists_5','team2_p_death_1', 'team2_p_death_2',
       'team2_p_death_3', 'team2_p_death_4', 'team2_p_death_5','team2_p_kills_1', 'team2_p_kills_2',
       'team2_p_kills_3', 'team2_p_kills_4', 'team2_p_kills_5','team2_score_1half',
       'team2_score_2half']

df_data[int_list] = df_data[int_list].astype(int)

float_list = ['team1_p_ADR_1','team1_p_ADR_1', 'team1_p_ADR_2', 'team1_p_ADR_3',
       'team1_p_ADR_4', 'team1_p_ADR_5','team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5','team1_p_rating_1', 'team1_p_rating_2', 'team1_p_rating_3',
       'team1_p_rating_4', 'team1_p_rating_5','team1_rating','team2_p_ADR_1', 'team2_p_ADR_2', 'team2_p_ADR_3',
       'team2_p_ADR_4', 'team2_p_ADR_5','team2_p_kast_1', 'team2_p_kast_2', 'team2_p_kast_3', 'team2_p_kast_4',
       'team2_p_kast_5','team2_p_rating_1', 'team2_p_rating_2', 'team2_p_rating_3',
       'team2_p_rating_4', 'team2_p_rating_5','team2_rating' ]

df_data[float_list] = df_data[float_list].astype(float, inplace=True)

In [149]:
df_data['team1main'].to_string()

MemoryError: 

In [147]:
df_data.dtypes

date                   datetime64[ns]
team1main                      object
team2main                      object
map                            object
eventmain                      object
link                           object
score_team1main                 int32
score_team2main                 int32
date_time                      object
map_number                      int32
match_type                     object
team1_clutches_won              int32
team1_first_kills               int32
team1_p_ADR_1                 float64
team1_p_ADR_2                 float64
team1_p_ADR_3                 float64
team1_p_ADR_4                 float64
team1_p_ADR_5                 float64
team1_p_asists_1                int32
team1_p_asists_2                int32
team1_p_asists_3                int32
team1_p_asists_4                int32
team1_p_asists_5                int32
team1_p_death_1                 int32
team1_p_death_2                 int32
team1_p_death_3                 int32
team1_p_deat

1. wyczszczenie nazw map
2. zrobienie tego ktory wygral lub zremisowal
3. wielkosc turnieju
4. date_time - zamien na rano wieczorem poludnie w nocy
5. zamiana score total na ?? diff map??
6. srednia_ratingow???  tu zaras sie cos wymysli
7. analiza ktory team na ktorym wygrywa czesciej - tzn ten co pierwszy z mapa\
8. średnie wyniki z ostatnich 10 spotkan dla kazdej z druzyn
9. srednie wyniki z ostaniego spotkania na tej mapie
10. srednie wyniki z ostatnich 10 spotkan na tej mapie
11. srednie wyniki ze spotkan 10 przeciwko
12. srednie wyniki ze spotkan na tej mapie wszystkie druzyny
13. srednie wyniki dla 1 strony, srednie wyniki dla 2 strony
14. srednie wyniki z pierwszej i 2 polowy ze wzgledu na zaczynana strone i mape


MOZE WYKORZYSTAC CZAS JAKO WAGE


PAMIETAJ o REQUIREMENTSACH

In [14]:
len(df_data.team1_player_name_1.unique())

5401

In [21]:
df_data[['score_team1main','team1main']].rolling(10, min_periods=1, on='team1main').sum()

,score_team1main,team1main
0,16.0,NiP
1,32.0,NiP
2,48.0,NiP
3,64.0,NiP
4,66.0,Red Dead
5,82.0,mousesports
6,98.0,mousesports
7,105.0,Tt Dragons
8,112.0,Tt Dragons
9,128.0,mousesports


In [31]:
df_data['score_team1main'][df_data['team1main'] == 'NiP'].shift(1).rolling(10).mean()

0          NaN
1          NaN
2          NaN
3          NaN
46         NaN
47         NaN
48         NaN
49         NaN
340        NaN
341        NaN
353       15.9
354       15.9
355       15.9
356       15.9
375       15.9
376       15.9
384       15.9
391       15.9
393       15.9
498       15.9
499       16.0
504       16.0
505       16.0
524       16.0
525       16.0
550       16.0
552       16.0
556       16.0
557       16.0
558       16.0
          ... 
141350    13.0
141351    13.1
141352    13.2
141455    14.0
141456    13.9
142427    12.7
142620    12.7
142621    12.7
142683    12.7
142684    12.5
142805    12.3
142806    11.9
142807    11.5
143389    11.1
143510    12.3
143534    13.5
143560    12.3
143561    12.3
143952    12.3
143953    12.5
144026    12.7
144027    13.4
144028    14.1
144464    14.8
144465    14.2
144466    13.6
144467    14.4
146439    14.0
146496    13.5
146497    12.5
Name: score_team1main, Length: 1405, dtype: float64

In [ ]:
df["mean"] = df["numeric_col"].shift(1).rolling(window=3).mean()

In [37]:
len(df_data['eventmain'].unique()) #dla eventu wez wytnij lata regular expresion - i potem sporobuj zobaczyc ile tego bedzie, wez majory do majorow reszte nizej

3073

In [38]:
print('nie umarl')

nie umarl


In [71]:
df_data[df_data.team2_score_2half.isna()]

,date,team1main,team2main,map,eventmain,link,score_team1main,score_team2main,date_time,eventdetails,map_number,match_type,score_team1details,score_team2details,score_total,team1details,team1_clutches_won,team1_first_kills,team1_p_ADR_1,team1_p_ADR_2,team1_p_ADR_3,team1_p_ADR_4,team1_p_ADR_5,team1_p_asists_1,team1_p_asists_2,team1_p_asists_3,team1_p_asists_4,team1_p_asists_5,team1_p_death_1,team1_p_death_2,team1_p_death_3,team1_p_death_4,team1_p_death_5,team1_p_kast_1,team1_p_kast_2,team1_p_kast_3,team1_p_kast_4,team1_p_kast_5,team1_p_kills_1,team1_p_kills_2,team1_p_kills_3,team1_p_kills_4,team1_p_kills_5,team1_p_rating_1,team1_p_rating_2,team1_p_rating_3,team1_p_rating_4,team1_p_rating_5,team1_player_name_1,team1_player_name_2,team1_player_name_3,team1_player_name_4,team1_player_name_5,team1_rating,team1_score_1half,team1_score_2half,team1side,team2details,team2_clutches_won,team2_first_kills,team2_p_ADR_1,team2_p_ADR_2,team2_p_ADR_3,team2_p_ADR_4,team2_p_ADR_5,team2_p_asists_1,team2_p_asists_2,team2_p_asists_3,team2_p_asists_4,team2_p_asists_5,team2_p_death_1,team2_p_death_2,team2_p_death_3,team2_p_death_4,team2_p_death_5,team2_p_kast_1,team2_p_kast_2,team2_p_kast_3,team2_p_kast_4,team2_p_kast_5,team2_p_kills_1,team2_p_kills_2,team2_p_kills_3,team2_p_kills_4,team2_p_kills_5,team2_p_rating_1,team2_p_rating_2,team2_p_rating_3,team2_p_rating_4,team2_p_rating_5,team2_player_name_1,team2_player_name_2,team2_player_name_3,team2_player_name_4,team2_player_name_5,team2_rating,team2_score_1half,team2_score_2half,map_name
406,2012-11-15,zNation,Tribute To Sylvester,Mirage_ce,SLTV StarSeries IV,https://www.hltv.org/stats/matches/mapstatsid/...,16,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
418,2012-11-14,Imaginary,BuyKey,Dust2_se,SLTV StarSeries IV,https://www.hltv.org/stats/matches/mapstatsid/...,16,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
435,2012-11-14,dAT,Tribute To Sylvester,Nuke_se,SLTV StarSeries IV,https://www.hltv.org/stats/matches/mapstatsid/...,13,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019,2013-02-21,KerchNET,VeryGames,Nuke_se,SLTV StarSeries V,https://www.hltv.org/stats/matches/mapstatsid/...,4,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1180,2013-02-26,VeryGames,mousesports,Inferno_se,SLTV StarSeries V,https://www.hltv.org/stats/matches/mapstatsid/...,16,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2299,2013-05-20,CPLAY,PyRoGEN,Dust2

In [49]:
df_data.columns

Index(['date', 'team1main', 'team2main', 'map', 'eventmain', 'link',
       'score_team1main', 'score_team2main', 'date_time', 'eventdetails',
       'map_number', 'match_type', 'score_team1details', 'score_team2details',
       'score_total', 'team1details', 'team1_clutches_won',
       'team1_first_kills', 'team1_p_ADR_1', 'team1_p_ADR_2', 'team1_p_ADR_3',
       'team1_p_ADR_4', 'team1_p_ADR_5', 'team1_p_asists_1',
       'team1_p_asists_2', 'team1_p_asists_3', 'team1_p_asists_4',
       'team1_p_asists_5', 'team1_p_death_1', 'team1_p_death_2',
       'team1_p_death_3', 'team1_p_death_4', 'team1_p_death_5',
       'team1_p_kast_1', 'team1_p_kast_2', 'team1_p_kast_3', 'team1_p_kast_4',
       'team1_p_kast_5', 'team1_p_kills_1', 'team1_p_kills_2',
       'team1_p_kills_3', 'team1_p_kills_4', 'team1_p_kills_5',
       'team1_p_rating_1', 'team1_p_rating_2', 'team1_p_rating_3',
       'team1_p_rating_4', 'team1_p_rating_5', 'team1_player_name_1',
       'team1_player_name_2', 'team1_pl